In [1]:
!pip install predictionguard
!pip install langdetect
!pip install langchain

In [2]:
pip show predictionguard

Name: predictionguard
Version: 2.5.0
Summary: Create controlled and compliant AI systems with PredictionGuard.
Home-page: 
Author: 
Author-email: Daniel Whitenack <dan@predictionguard.com>
License: 
Location: D:\Anaconda\Lib\site-packages
Requires: requests, tabulate
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import random

from predictionguard import PredictionGuard
from getpass import getpass
import pandas as pd
from sklearn.model_selection import train_test_split
from langdetect import detect
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import numpy as np
import time
from datetime import datetime


In [4]:
data = pd.read_csv('examples.csv')

In [5]:
data.head()

,Id,q1,q2,q3,q4,q5,q6,q7,q8,q9,...,q14,q15,q16,q17,q18,q19,q20,q21,q22,label
0,bfe3aa97-eb08-43d9-88a5-34ce93509321,6,6.0,1,2,2,2,2,2,1,...,2,2,1,2,1,2,1,"As a surviving child, the weekend seminar with...","""As a surviver chil going to the TAPS Seminar,...",3
1,3b47ee28-e621-4497-949b-0fa0a59044f7,0,4.0,1,2,2,2,0,1,1,...,2,2,0,1,1,2,2,NaN,Como hijo sobreviviente que asistió al Seminar...,3
2,e21808f4-2c7f-4cdf-90f5-daa7ccccc843,0,5.0,2,2,2,2,0,4,2,...,1,2,2,2,2,2,2,NaN,"""The TAPS Seminar was incredibly valuable for ...",4
3,59385c6a-5a11-4348-b0d1-c1f87b56cbd3,5,NaN,0,1,1,1,4,2,1,...,1,3,2,3,3,3,1,NaN,"""I attended the TAPS Seminar recently, and it ...",2
4,6a769ce6-900b-4490-9d67-6a9e8190b78c,0,0.0,1,2,2,2,2,2,1,...,1,2,2,1,1,2,1,"During our unforgettable weekend, we cherishin...",NaN,3


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 24 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      50 non-null     object 
 1   q1      50 non-null     int64  
 2   q2      34 non-null     float64
 3   q3      50 non-null     int64  
 4   q4      50 non-null     int64  
 5   q5      50 non-null     int64  
 6   q6      50 non-null     int64  
 7   q7      50 non-null     int64  
 8   q8      50 non-null     int64  
 9   q9      50 non-null     int64  
 10  q10     50 non-null     int64  
 11  q11     50 non-null     int64  
 12  q12     50 non-null     int64  
 13  q13     50 non-null     int64  
 14  q14     50 non-null     int64  
 15  q15     50 non-null     int64  
 16  q16     50 non-null     int64  
 17  q17     50 non-null     int64  
 18  q18     50 non-null     int64  
 19  q19     50 non-null     int64  
 20  q20     50 non-null     int64  
 21  q21     29 non-null     object 
 22  q22 

In [7]:
data.isnull().sum()

Id        0
q1        0
q2       16
q3        0
q4        0
q5        0
q6        0
q7        0
q8        0
q9        0
q10       0
q11       0
q12       0
q13       0
q14       0
q15       0
q16       0
q17       0
q18       0
q19       0
q20       0
q21      21
q22      28
label     0
dtype: int64

In [8]:
pg_access_token = getpass('Enter your Prediction Guard access api key: ')
os.environ['PREDICTIONGUARD_API_KEY'] = pg_access_token

Enter your Prediction Guard access api key: ········


In [9]:
client = PredictionGuard()

In [10]:
data.label.value_counts()

label
2    11
3    10
4     9
6     7
1     7
5     6
Name: count, dtype: int64

In [11]:
# Function to format a single row without including the Id in the response dictionary
def format_response(row):
    formatted_response = {}
    for question, response in row.items():
        if question != "Id":  # Exclude the Id field from the response
            formatted_response[question] = response
    return formatted_response

In [12]:
data['formatted_response'] = data.drop(columns = 'label').apply(format_response, axis=1)

In [13]:
examples = """
 --------------
Q1:    3
Q2:    nan
Q3:    1
Q4:    2
Q5:    2
Q6:    2
Q7:    0
Q8:    1
Q9:    1
Q10:   2
Q11:   1
Q12:   1
Q13:   2
Q14:   2
Q15:   2
Q16:   1
Q17:   1
Q18:   1
Q19:   1
Q20:   1
Q21:   "I had a breakthrough moment at the TAPS seminar when we participated in a group activity that encouraged us to express our emotions and share our experiences with other surviving children. It was amazing to feel the connection and understanding with others who are going through a similar journey. Our facilitator created a safe and supportive environment which allowed me to genuinely open up about my grief and learn tools to process my emotions healthily. This moment was deeply impactful for me and my child, and it has given us the strength to continue forward in our healing process."
Q22:   "As a surviving child, attending the TAPS Seminar was incredibly helpful in helping me learn how to process my grief. It provided a safe and supportive environment for me to open up, share my emotions, and connect with others who are going through a similar experience. The seminars and workshops not only allowed me to gain valuable tools and resources for managing my grief but also taught me the importance of normalizing these emotions with my peers. I appreciated the opportunity to participate in age-appropriate activities for kids, which helped me feel understood and supported. Overall, this weekend was a wonderful source of guidance and acknowledgment, and I am grateful for the invaluable connections and lessons that I have gained from the TAPS Seminar."
Label: 3
Reason: Labeled as Stage 3 due to the emphasis on learning to process grief, gaining tools and resources, and finding support in a safe environment, which aligns with developing healthy coping mechanisms and normalizing emotions in the grieving process.

Q1:    0
Q2:    nan
Q3:    2
Q4:    2
Q5:    2
Q6:    2
Q7:    0
Q8:    1
Q9:    2
Q10:   2
Q11:   1
Q12:   2
Q13:   2
Q14:   0
Q15:   2
Q16:   2
Q17:   2
Q18:   2
Q19:   2
Q20:   2
Q21:   nan
Q22:   nan
Label: 4
Reason: Labeled as Stage 4 due to the consistent positive responses across questions related to processing grief and finding strength, suggesting a focus on navigating meaningful moments and sustaining support within the family or community, typical of "Moments That Matter."

Q1:    0
Q2:    nan
Q3:    0
Q4:    0
Q5:    0
Q6:    0
Q7:    1
Q8:    0
Q9:    3
Q10:   3
Q11:   1
Q12:   3
Q13:   3
Q14:   3
Q15:   0
Q16:   3
Q17:   3
Q18:   0
Q19:   3
Q20:   3
Q21:   "I am truly grateful for the opportunity to attend the TAPS seminar and connect with others who share in our grief. My favorite moment of the weekend was during the panel discussion on coping with loss, where I found both inspiration and solidarity in hearing the experiences of others. It was a turning point for me, realizing the power of community support and understanding that I am not alone in this journey.
I had an incredible breakthrough moment during the art therapy session, where I found solace in expressing my emotions through painting. It allowed me to confront my grief in a way that was comforting and healing. I believe this experience has prepared me to give back to the TAPS community by offering support and guidance to those who may be struggling with their own journey. I am eager to participate in mentorship programs, charity drives, and events, and contribute to making a difference in the lives of others."
Q22:   nan
Label: 6
Reason: Labeled as Stage 6 due to the respondent's focus on giving back to the TAPS community, a readiness to support others, and participation in mentorship and charity, indicating a commitment to help others as part of their healing journey.

Q1:    0
Q2:    4.0
Q3:    1
Q4:    2
Q5:    2
Q6:    2
Q7:    0
Q8:    1
Q9:    1
Q10:   2
Q11:   1
Q12:   1
Q13:   2
Q14:   2
Q15:   2
Q16:   0
Q17:   1
Q18:   1
Q19:   2
Q20:   2
Q21:   nan
Q22:   Como hijo sobreviviente que asistió al Seminario TAPS, encontré increíblemente beneficioso estar rodeado de personas que están pasando por una experiencia similar. Fue reconfortante saber que no estoy solo en mi duelo y que hay otros niños de mi edad que entienden lo que estoy viviendo. El seminario me proporcionó un espacio seguro para procesar abiertamente mis emociones, y fue empoderador aprender herramientas y recursos para enfrentar la pérdida de mi familiar que sirvió en las fuerzas armadas. Estimo la orientación ofrecida por expertos en duelo y trauma, así como la oportunidad de normalizar mis sentimientos con otros que han pasado por lo mismo. Recomendaría altamente los Seminarios TAPS para otros familiares supervivientes; fue una experiencia transformadora que me permitió comprender y gestionar mejor mi viaje por el duelo.
Label: 3
Reason: Labeled as Stage 3 because the response emphasizes the benefit of a safe space for processing emotions, gaining tools to manage grief, and normalizing feelings through shared experiences, aligning with learning to process grief.

Q1:    4
Q2:    0.0
Q3:    1
Q4:    0
Q5:    0
Q6:    0
Q7:    2
Q8:    2
Q9:    0
Q10:   3
Q11:   1
Q12:   2
Q13:   3
Q14:   2
Q15:   1
Q16:   2
Q17:   0
Q18:   2
Q19:   0
Q20:   3
Q21:   nan
Q22:   "As a participant at the TAPS seminar, I appreciated the opportunity to be surrounded by individuals who understand the unique struggles of losing a loved one in the military. It was helpful to connect with others who are experiencing similar feelings of grief and trauma. However, I encountered some tension with my family members while attending the seminar. They seem to not understand the depth of my grief, and it is challenging to communicate that to them.
The TAPS seminar provided valuable resources for me to better support myself and my grieving process. The peer-based support and expert-led workshops offered a safe space for processing emotions and learning tools to manage my grief. While my family was not present during the seminar, I hope that the skills and connections I made can strengthen my ability to navigate the grieving process with them and provide support and understanding within our family unit.
I would like to request more opportunities to connect with other seminar attendees, as I believe building a supportive network can provide ongoing comfort and understanding. Additionally, it would be beneficial to have access to more family-oriented workshops or workshops tailored to addressing family dynamics and communication during grief."
Label: 2
Reason: Labeled as Stage 2 due to the focus on family-related challenges, including difficulties in communicating grief depth to family members and a desire for family-oriented support, which aligns with navigating family dynamics in the grieving process.

Q1:    0
Q2:    5.0
Q3:    2
Q4:    2
Q5:    2
Q6:    2
Q7:    0
Q8:    4
Q9:    2
Q10:   2
Q11:   1
Q12:   2
Q13:   2
Q14:   1
Q15:   2
Q16:   2
Q17:   2
Q18:   2
Q19:   2
Q20:   2
Q21:   nan
Q22:   "The TAPS Seminar was incredibly valuable for me during this time of renewed grief experiences. The weekend provided a safe space for me to connect with others who understand my unique loss, and the resources offered by the grief and trauma experts were incredibly helpful in navigating the complexities of my journey. One particular aspect that stood out to me was the focus on navigating special moments with my surviving family unit. It gave me helpful strategies to ensure that we can celebrate important milestones and moments of accomplishment together while honoring our loved one's memory. Thank you for the opportunity to be a part of such a supportive and understanding community."
Label: 4
Reason: Labeled as Stage 4 because the response highlights the importance of navigating significant milestones and moments with family, reflecting a focus on shared grief experiences during "moments that matter," which is characteristic of this stage.

Q1:    1
Q2:    nan
Q3:    1
Q4:    2
Q5:    2
Q6:    1
Q7:    1
Q8:    1
Q9:    1
Q10:   1
Q11:   3
Q12:   0
Q13:   1
Q14:   4
Q15:   0
Q16:   1
Q17:   1
Q18:   0
Q19:   1
Q20:   0
Q21:   "Although it's been a challenging few days, I must admit that my favorite moment of the weekend was when we had the group activity where everyone was encouraged to express their feelings through art. It was truly a cathartic experience, and I found it comforting to share my emotions through painting and drawing alongside others who were going through similar heartaches. It felt like we were all part of a healing community, and I could feel a sense of unity that I hadn't experienced before during my grief journey.
It was during that art session when I had a breakthrough moment. I realized that I don't have to navigate this difficult time alone. I met an adult who shared their own experience as a survivor's child and found ways to create a new sense of purpose in their life. Their story inspired me, and I see now that we can find meaning in honoring our loved ones in our own ways while also embracing our individual growth. I am grateful for the connections and resources that TAPS provided this weekend, as they have given me hope for the future."
Q22:   At TAPS Seminar, I connected with fellow survivors, gained emotional support and practical resources, but still struggle to find purpose daily as a surviving child; more tailored guidance is needed for young grieving family members.
Label: 1
Reason: Labeled as Stage 1 due to the respondent’s expression of intense emotions, a feeling of isolation, and the struggle to find daily purpose, which aligns with the initial shock and overwhelming grief of immediate loss.

Q1:    3
Q2:    nan
Q3:    1
Q4:    2
Q5:    2
Q6:    0
Q7:    2
Q8:    2
Q9:    2
Q10:   2
Q11:   1
Q12:   1
Q13:   3
Q14:   1
Q15:   1
Q16:   2
Q17:   1
Q18:   1
Q19:   1
Q20:   2
Q21:   "I had a very emotional yet healing experience during the support group session for children. The counselor created a safe and welcoming atmosphere for all the kids to share their feelings and experiences openly. We played an interactive game that allowed us to understand the importance of acknowledging and processing our emotions together. I observed my own child as well as other children in the group opening up, and it was a rewarding moment for me as a parent. This normalized our grief and gave me hope that, together, we can continue to heal and grow."
Q22:   nan
Label: 3
Reason: Labeled as Stage 3 because the response emphasizes the process of acknowledging and managing emotions in a supportive setting, which aligns with learning to process grief and gaining hope through shared healing experiences.

Q1:    3
Q2:    nan
Q3:    1
Q4:    2
Q5:    2
Q6:    2
Q7:    2
Q8:    1
Q9:    1
Q10:   1
Q11:   2
Q12:   1
Q13:   2
Q14:   2
Q15:   2
Q16:   2
Q17:   1
Q18:   2
Q19:   2
Q20:   2
Q21:   The last kids meetin' was reely wun-hoanahlin' - so many powerful feelyings shared as the guider helped the little ones open up thur art. So proud to see ma baby speekin' thur paintbrushes and hearin' other kiddos' brav' stories. It made me feel somethin' reely deep innah my heart. That time chaynged how I do parenting for the bettah, since I got a lot of new ideas. Itz impawtant to teach my baby to feel things and share them with others. This experiance wuz truely gratefur by TAPS, they helped me see the best ways to hep ma baby. They left somethin' wunneh in my heart as a mama and I'll try to do bettah with that in mind, foah-evah.
Q22:   nan
Label: 3
Reason: Labeled as Stage 3 due to the emphasis on processing emotions, learning new ways to support a child through grief, and the value of emotional expression, which aligns with learning to process and manage grief.

Q1:    1
Q2:    3.0
Q3:    1
Q4:    1
Q5:    2
Q6:    1
Q7:    1
Q8:    1
Q9:    0
Q10:   1
Q11:   4
Q12:   0
Q13:   1
Q14:   3
Q15:   1
Q16:   0
Q17:   0
Q18:   0
Q19:   1
Q20:   0
Q21:   "During the weekend seminar, my favorite moment was when we were encouraged to share our stories with each other. It felt like a beacon of light in my otherwise dark and overwhelming journey. We were all struggling, and there was a certain comfort in knowing that we're not alone in this. It's not easy to handle everything by myself, especially as a surviving child. I feel deeply disconnected without guidance and attention from my grieving adults, but sharing my experiences with everyone at TAPS gave me some sense of purpose and connection. Though I still face many challenges, this weekend has reminded me that we're all in this together, and that is what I find immensely comforting."
Q22:   nan
Label: 1
Reason: Labeled as Stage 1 due to the respondent’s expression of feeling overwhelmed, isolated, and disconnected, with a strong need for support and guidance, which aligns with the intense, initial shock and emotional state of immediate grief.

Q1:    4
Q2:    nan
Q3:    1
Q4:    1
Q5:    1
Q6:    0
Q7:    0
Q8:    3
Q9:    3
Q10:   0
Q11:   0
Q12:   0
Q13:   0
Q14:   3
Q15:   0
Q16:   1
Q17:   1
Q18:   0
Q19:   1
Q20:   2
Q21:   nan
Q22:   nan
Label: 2
Reason: Labeled as Stage 2 due to the low scores in optimism, inner strength, and direction, combined with high anxiety about the future, suggesting struggles with family relationships and support, characteristic of navigating family dynamics in grief.

Q1:    0
Q2:    nan
Q3:    2
Q4:    2
Q5:    2
Q6:    2
Q7:    0
Q8:    0
Q9:    2
Q10:   2
Q11:   1
Q12:   2
Q13:   2
Q14:   1
Q15:   1
Q16:   2
Q17:   3
Q18:   2
Q19:   1
Q20:   2
Q21:   "I am grateful for the opportunities provided at TAPS to reminisce on the good times with my loved one and create new memories with my Family Unit. One moment that stands out for me was the candle lighting ceremony on the last night. As we lit our candles and shared our stories, I felt an overwhelming sense of unity and support from the other families present. This experience gave me the strength to face the upcoming tough days and find solace in our shared grief journey. We also had a breakthrough moment when we participated in the art therapy session. Expressing our emotions through painting and drawing helped us to better understand our individual grief experiences, while bringing us closer as a Family Unit. Overall, the weekend gave me hope and renewed energy to face life's challenges and cherish the moments that matter with my family."
Q22:   nan
Label: 4
Reason: Labeled as Stage 4 due to the focus on shared grief experiences during significant events, such as the candle lighting ceremony and art therapy, which brought unity and strength in navigating "moments that matter" with family.

Q1:    2
Q2:    7.0
Q3:    1
Q4:    2
Q5:    1
Q6:    2
Q7:    0
Q8:    1
Q9:    2
Q10:   2
Q11:   2
Q12:   1
Q13:   1
Q14:   3
Q15:   2
Q16:   2
Q17:   1
Q18:   2
Q19:   1
Q20:   1
Q21:   nan
Q22:   Asistir al Seminario TAPS junto a mi hijo fue una experiencia transformadora. Proporcionó un ambiente de apoyo para que procesáramos nuestro duelo, ofreciendo recursos valiosos, orientación y conexiones con otras personas enfrentando desafíos similares. El seminario mejoró significativamente nuestro viaje de sanación y altamente lo recomiendo.
Label: 3
Reason: Labeled as Stage 3 due to the response’s focus on processing grief, receiving support, and using resources and connections to aid in healing, aligning with learning to process emotions.

Q1:    0
Q2:    7.0
Q3:    2
Q4:    0
Q5:    1
Q6:    0
Q7:    3
Q8:    0
Q9:    3
Q10:   3
Q11:   0
Q12:   0
Q13:   3
Q14:   0
Q15:   3
Q16:   3
Q17:   3
Q18:   3
Q19:   3
Q20:   3
Q21:   At a recent TAPS seminar, a panel discussion of grieving family members sparked a breakthrough, inspiring me to support and uplift others in their grief journey, volunteer, and contribute to the TAPS community's mission of providing love, support, and healing.
Q22:   nan
Label: 6
Reason: Labeled as Stage 6 due to the response's focus on giving back, supporting others, and contributing to the TAPS community, reflecting a desire to help and mentor as part of personal healing.

Q1:    0
Q2:    nan
Q3:    2
Q4:    1
Q5:    1
Q6:    2
Q7:    1
Q8:    0
Q9:    3
Q10:   3
Q11:   0
Q12:   1
Q13:   3
Q14:   0
Q15:   1
Q16:   3
Q17:   3
Q18:   3
Q19:   3
Q20:   3
Q21:   At the TAPS seminar, I experienced a life-changing weekend through art-based group activities and a memorial service, which inspired me to create a local support group for grieving military families.
Q22:   The TAPS Seminar significantly impacted my life by offering valuable support, guidance, and a sense of community for survivors like me. Through meaningful connections and workshops, I discovered renewed purpose, strengthened my support system, and look forward to my continued growth with the TAPS family.
Label: 5
Reason: Labeled as Stage 5 due to the response's focus on finding community support, a renewed sense of purpose, and commitment to personal growth within the TAPS community.

Q1:    4
Q2:    nan
Q3:    2
Q4:    2
Q5:    0
Q6:    1
Q7:    3
Q8:    0
Q9:    1
Q10:   2
Q11:   3
Q12:   3
Q13:   2
Q14:   3
Q15:   3
Q16:   3
Q17:   3
Q18:   1
Q19:   0
Q20:   1
Q21:   "I had a breakthrough moment at the TAPS seminar that truly stands out for me. During one of the group discussions, I finally expressed my feelings of frustration and sadness to my fellow survivors. It was in that moment of vulnerability that I realized I wasn't alone in my struggles. We all shared similar experiences, and that brought a sense of relief and comfort to me. It also helped me to understand that my family members are also going through their unique grief journeys.
The support and understanding from the other seminar participants allowed me to see things from their perspective and have a more empathetic approach when interacting with them. As a result, I feel like the tension within our family has eased a bit, and we're learning how to support each other better. I am grateful for the opportunity to connect with others who truly understand what we are going through as survivors. This seminar and its community will always hold a special place in my heart."
Q22:   "I appreciated the TAPS Seminar as it provided a safe space for me to process my grief and connect with others who have gone through similar experiences. However, I found the lack of support from some family members during the seminar difficult to navigate. It seems that they were not at the same stage of grief as me, and this created tension among us. While the TAPS team offered valuable tools and resources for coping with grief, I believe they could have provided more guidance on addressing these underlying family dynamics. Overall, though, my experience at the seminar was insightful and I am grateful for the support of my fellow seminar attendees who helped me during this challenging journey."
Label: 2

Q1:    6
Q2:    nan
Q3:    0
Q4:    2
Q5:    1
Q6:    2
Q7:    0
Q8:    3
Q9:    3
Q10:   3
Q11:   3
Q12:   1
Q13:   3
Q14:   2
Q15:   0
Q16:   3
Q17:   1
Q18:   1
Q19:   0
Q20:   2
Q21:   "In sessions promoting open communitcation, we learnt to acknowledge and undarstand our unique grief experienses, finding support with each other whilst celebrating our love ones' legacies during the remembrance ceremony, leading to uniting and sharid healing."
Q22:   "I attended the TAPS Seminar, and overall, I found the experience to be both thought-provoking and supportive. It provided me with valuable tools and resources to help navigate my grief journey. However, one area where I felt some challenges was in the dynamics within my family unit. While I was able to connect with other attendees and receive their empathetic understanding, I discovered differing grief experiences and perceptions among my family members.
I appreciated the opportunity to attend workshops and activities focused on grief and self-care, as it allowed me to gain a better understanding of my own emotions and needs. However, I noticed that the lack of support from some family members made it harder for me to cope with the emotional aspects of the seminar. It highlighted the ongoing tension within our family and the continued need for open communication and understanding.
Despite this challenge, I am grateful for the connections I made during the seminar and the valuable resources provided by TAPS experts. I hope to continue engaging with the TAPS community, as I believe it has the potential to help strengthen the support system within my family and further my own healing process."
Label: 2
Reason: Labeled as Stage 2 due to emphasis on family dynamics, differing grief experiences, and need for open communication.

Q1:    3
Q2:    1.0
Q3:    1
Q4:    1
Q5:    1
Q6:    1
Q7:    2
Q8:    2
Q9:    0
Q10:   1
Q11:   3
Q12:   1
Q13:   0
Q14:   3
Q15:   0
Q16:   0
Q17:   0
Q18:   0
Q19:   2
Q20:   1
Q21:   nan
Q22:   nan
Label: 1
Reason:It reflects intense feelings of isolation, fear about the future, and an absence of positivity or purpose, aligning with the overwhelmed and uncertain state characteristic of immediate grief.

Q1:    1
Q2:    7.0
Q3:    2
Q4:    1
Q5:    1
Q6:    2
Q7:    1
Q8:    0
Q9:    3
Q10:   3
Q11:   0
Q12:   3
Q13:   3
Q14:   0
Q15:   3
Q16:   3
Q17:   3
Q18:   3
Q19:   3
Q20:   3
Q21:   Example response: "Attending the TAPS seminar over the weekend provided me with a sense of belonging and strengthened my connection to the surviving family unit. My favorite moment came during the group discussion, where we shared our stories and experiences. We were all in a similar situation, and I felt a deep sense of comfort knowing that I'm not alone in this journey.

The breakthrough moment for me was when the trauma expert emphasized the importance of finding new purpose and setting goals to help us move towards positive integration. This resonated with me, and I realized that I need to focus on my own personal growth and well-being in order to continue honoring the memory of my loved one. This weekend has given me the motivation and support to find that new purpose and connect with this wonderful community. Thank you, TAPS, for providing this invaluable service."
Q22:   nan
Label: 5
Reason: This response is classified as Stage 5 becuase in the open-ended questions, despite mentioning of connections and a new sense of purpose, the respondent does not mention giving back or readiness to support others.

Q1:    7
Q2:    1.0
Q3:    2
Q4:    1
Q5:    2
Q6:    2
Q7:    0
Q8:    2
Q9:    3
Q10:   3
Q11:   0
Q12:   3
Q13:   3
Q14:   0
Q15:   3
Q16:   3
Q17:   3
Q18:   3
Q19:   3
Q20:   3
Q21:   Durante el seminario de fin de semana, tuve una experiencia profunda y emocional al conectarme con otras familias que también navegaban por los desafíos del duelo y la pérdida. La discusión en panel sobre redescubrir propósito y sentido en el after de la pérdida realmente me resonó, y me ofreció valiosos insights que me animaron a reflexionar sobre la dirección de mi propia vida. Esta discusión no solo resaltó la importancia de forjar conexiones con aquellos que comparten y entienden nuestras experiencias, sino que también me impulsó a considerar adoptar una nueva perspectiva en mi viaje.
Uno de los puntos de inflexión del seminario fue el taller sobre crear un legado viviente para nuestros seres queridos. A través de este taller, obtuve claridad y motivación renovada para establecer un proyecto comunitario dedicado a honrar la memoria de mi ser amado de una manera significativa y positiva. Al trabajar en colaboración con individuos que comparten un viaje similar, podemos apoyarnos mutuamente, crear un espacio seguro para conversaciones emotivas y garantizar que la memoria de nuestros seres queridos continúe brillando en nuestras vidas y en la vida de los demás.
Como resultado de mi participación en este seminario de fin de semana, he desarrollado un vínculo indisolvable con la comunidad TAPS y estoy increíblemente agradecido por su apoyo inquebrantable y las enriquecedoras perspectivas que me han compartido. Este seminario ha abierto mis ojos a nuevas oportunidades de crecimiento, sanación y conexión, y estoy emocionado por emprender este viaje con mi comunidad recién formada.
Q22:   "Encontré el Seminario TAPS como una experiencia realmente transformadora. Me proporcionó un sentimiento de conexión y pertenencia dentro de una comunidad que entiende mi dolor único. Los talleres y recursos ofrecidos me han dado la fuerza y la orientación para emprender mi viaje hacia una integración positiva. No solo adquirí herramientas para gestionar mi duelo, sino que también establecí conexiones significativas con personas que tienen una historia similar. Estoy agradecido por el apoyo y la validación que recibí en el seminario, y me siento empoderado para seguir adelante con un nuevo sentimiento de propósito."
Label: 5
Reason: This response is classified as Stage 5 becuase in the open-ended questions, despite mentioning of connections and a new sense of purpose, the respondent does not mention giving back or readiness to support others.

Q1:    0
Q2:    7.0
Q3:    2
Q4:    0
Q5:    1
Q6:    1
Q7:    1
Q8:    0
Q9:    3
Q10:   3
Q11:   0
Q12:   3
Q13:   3
Q14:   0
Q15:   3
Q16:   3
Q17:   3
Q18:   3
Q19:   3
Q20:   3
Q21:   nan
Q22:   "I went to TAPS seminar becuz I am a healthy n proactive partipicipant in the grief journey. Duaring the weekend, it helped me a lotta to connect wit othr survivors, we talked bout our experenses. I tink TAPS combo of peer support and expert guidens is uniq n valuab to heeling.
I left the seminar feelin empowerd, ready to help in TAPS communti. I wanna teech, volunteeer in charity drives n evnts and lend a hand. It is impotant to giv bak to TAPS is pa of my mouring process. I belive it can help TAPS gro and wel comtunity.
Ovaoll, I strongli recomd TAPS seminar to any1 who is grieving the los of a loved one that servd in US militry. The exprience was lif chanjin for me. I hop by talkin bout my stori n engagin othr partipicpants, I can helot her find comfort n communit. It did for me."
Label: 6
Reason: Labeled as Stage 6 due to the respondent's focus on giving back, volunteering, and supporting others in the TAPS community, showing a proactive approach to help and empower others as part of their healing journey.---------------------
"""

In [14]:
# examples = """
#  --------------
# Q1:    3
# Q2:    nan
# Q3:    1
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    1
# Q9:    1
# Q10:   2
# Q11:   1
# Q12:   1
# Q13:   2
# Q14:   2
# Q15:   2
# Q16:   1
# Q17:   1
# Q18:   1
# Q19:   1
# Q20:   1
# Q21:   "I had a breakthrough moment at the TAPS seminar when we participated in a group activity that encouraged us to express our emotions and share our experiences with other surviving children. It was amazing to feel the connection and understanding with others who are going through a similar journey. Our facilitator created a safe and supportive environment which allowed me to genuinely open up about my grief and learn tools to process my emotions healthily. This moment was deeply impactful for me and my child, and it has given us the strength to continue forward in our healing process."
# Q22:   "As a surviving child, attending the TAPS Seminar was incredibly helpful in helping me learn how to process my grief. It provided a safe and supportive environment for me to open up, share my emotions, and connect with others who are going through a similar experience. The seminars and workshops not only allowed me to gain valuable tools and resources for managing my grief but also taught me the importance of normalizing these emotions with my peers. I appreciated the opportunity to participate in age-appropriate activities for kids, which helped me feel understood and supported. Overall, this weekend was a wonderful source of guidance and acknowledgment, and I am grateful for the invaluable connections and lessons that I have gained from the TAPS Seminar."
# Label: 3
# Reason: Labeled as Stage 3 due to the emphasis on learning to process grief, gaining tools and resources, and finding support in a safe environment, which aligns with developing healthy coping mechanisms and normalizing emotions in the grieving process.

# Q1:    0
# Q2:    nan
# Q3:    2
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    1
# Q9:    2
# Q10:   2
# Q11:   1
# Q12:   2
# Q13:   2
# Q14:   0
# Q15:   2
# Q16:   2
# Q17:   2
# Q18:   2
# Q19:   2
# Q20:   2
# Q21:   nan
# Q22:   nan
# Label: 4
# Reason: Labeled as Stage 4 due to the consistent positive responses across questions related to processing grief and finding strength, suggesting a focus on navigating meaningful moments and sustaining support within the family or community, typical of "Moments That Matter."

# Q1:    0
# Q2:    nan
# Q3:    0
# Q4:    0
# Q5:    0
# Q6:    0
# Q7:    1
# Q8:    0
# Q9:    3
# Q10:   3
# Q11:   1
# Q12:   3
# Q13:   3
# Q14:   3
# Q15:   0
# Q16:   3
# Q17:   3
# Q18:   0
# Q19:   3
# Q20:   3
# Q21:   "I am truly grateful for the opportunity to attend the TAPS seminar and connect with others who share in our grief. My favorite moment of the weekend was during the panel discussion on coping with loss, where I found both inspiration and solidarity in hearing the experiences of others. It was a turning point for me, realizing the power of community support and understanding that I am not alone in this journey.
# I had an incredible breakthrough moment during the art therapy session, where I found solace in expressing my emotions through painting. It allowed me to confront my grief in a way that was comforting and healing. I believe this experience has prepared me to give back to the TAPS community by offering support and guidance to those who may be struggling with their own journey. I am eager to participate in mentorship programs, charity drives, and events, and contribute to making a difference in the lives of others."
# Q22:   nan
# Label: 6
# Reason: Labeled as Stage 6 due to the respondent's focus on giving back to the TAPS community, a readiness to support others, and participation in mentorship and charity, indicating a commitment to help others as part of their healing journey.

# Q1:    0
# Q2:    4.0
# Q3:    1
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    1
# Q9:    1
# Q10:   2
# Q11:   1
# Q12:   1
# Q13:   2
# Q14:   2
# Q15:   2
# Q16:   0
# Q17:   1
# Q18:   1
# Q19:   2
# Q20:   2
# Q21:   nan
# Q22:   Como hijo sobreviviente que asistió al Seminario TAPS, encontré increíblemente beneficioso estar rodeado de personas que están pasando por una experiencia similar. Fue reconfortante saber que no estoy solo en mi duelo y que hay otros niños de mi edad que entienden lo que estoy viviendo. El seminario me proporcionó un espacio seguro para procesar abiertamente mis emociones, y fue empoderador aprender herramientas y recursos para enfrentar la pérdida de mi familiar que sirvió en las fuerzas armadas. Estimo la orientación ofrecida por expertos en duelo y trauma, así como la oportunidad de normalizar mis sentimientos con otros que han pasado por lo mismo. Recomendaría altamente los Seminarios TAPS para otros familiares supervivientes; fue una experiencia transformadora que me permitió comprender y gestionar mejor mi viaje por el duelo.
# Label: 3
# Reason: Labeled as Stage 3 because the response emphasizes the benefit of a safe space for processing emotions, gaining tools to manage grief, and normalizing feelings through shared experiences, aligning with learning to process grief.

# Q1:    4
# Q2:    0.0
# Q3:    1
# Q4:    0
# Q5:    0
# Q6:    0
# Q7:    2
# Q8:    2
# Q9:    0
# Q10:   3
# Q11:   1
# Q12:   2
# Q13:   3
# Q14:   2
# Q15:   1
# Q16:   2
# Q17:   0
# Q18:   2
# Q19:   0
# Q20:   3
# Q21:   nan
# Q22:   "As a participant at the TAPS seminar, I appreciated the opportunity to be surrounded by individuals who understand the unique struggles of losing a loved one in the military. It was helpful to connect with others who are experiencing similar feelings of grief and trauma. However, I encountered some tension with my family members while attending the seminar. They seem to not understand the depth of my grief, and it is challenging to communicate that to them.
# The TAPS seminar provided valuable resources for me to better support myself and my grieving process. The peer-based support and expert-led workshops offered a safe space for processing emotions and learning tools to manage my grief. While my family was not present during the seminar, I hope that the skills and connections I made can strengthen my ability to navigate the grieving process with them and provide support and understanding within our family unit.
# I would like to request more opportunities to connect with other seminar attendees, as I believe building a supportive network can provide ongoing comfort and understanding. Additionally, it would be beneficial to have access to more family-oriented workshops or workshops tailored to addressing family dynamics and communication during grief."
# Label: 2
# Reason: Labeled as Stage 2 due to the focus on family-related challenges, including difficulties in communicating grief depth to family members and a desire for family-oriented support, which aligns with navigating family dynamics in the grieving process.

# Q1:    0
# Q2:    5.0
# Q3:    2
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    4
# Q9:    2
# Q10:   2
# Q11:   1
# Q12:   2
# Q13:   2
# Q14:   1
# Q15:   2
# Q16:   2
# Q17:   2
# Q18:   2
# Q19:   2
# Q20:   2
# Q21:   nan
# Q22:   "The TAPS Seminar was incredibly valuable for me during this time of renewed grief experiences. The weekend provided a safe space for me to connect with others who understand my unique loss, and the resources offered by the grief and trauma experts were incredibly helpful in navigating the complexities of my journey. One particular aspect that stood out to me was the focus on navigating special moments with my surviving family unit. It gave me helpful strategies to ensure that we can celebrate important milestones and moments of accomplishment together while honoring our loved one's memory. Thank you for the opportunity to be a part of such a supportive and understanding community."
# Label: 4
# Reason: Labeled as Stage 4 because the response highlights the importance of navigating significant milestones and moments with family, reflecting a focus on shared grief experiences during "moments that matter," which is characteristic of this stage.

# Q1:    5
# Q2:    5.0
# Q3:    0
# Q4:    0
# Q5:    2
# Q6:    1
# Q7:    2
# Q8:    4
# Q9:    1
# Q10:   2
# Q11:   3
# Q12:   1
# Q13:   0
# Q14:   1
# Q15:   2
# Q16:   2
# Q17:   3
# Q18:   2
# Q19:   1
# Q20:   0
# Q21:   "Aprecié la oportunidad de conocer a otros que comparten una experiencia similar y el fin de semana me permitió empáticamente entender diferentes perspectivas. Mi momento favorito ocurrió durante una actividad en grupo, donde éramos animados a expresar nuestras emociones sin juicio. Aunque generó cierta tensión dentro de la unidad familiar, sentí en persona escuchado y comprendido por los demás en el seminario. Esta experiencia me enseñó a ser más paciente y aceptante de nuestros viajes de duelo únicos en familia. Me di cuenta de que nuestro proceso de sanación colectiva es esencial para que nos encontremos adelante. Romper el hielo con otras familias me ha dado esperanza e inspiración para apoyarnos mutuamente en este camino."
# Q22:   nan
# Label: 2
# Reason: Labeled as Stage 2 due to the emphasis on family dynamics, understanding different grief perspectives within the family, and fostering patience and support, which aligns with navigating family relationships in the grieving process.

# Q1:    1
# Q2:    nan
# Q3:    1
# Q4:    2
# Q5:    2
# Q6:    1
# Q7:    1
# Q8:    1
# Q9:    1
# Q10:   1
# Q11:   3
# Q12:   0
# Q13:   1
# Q14:   4
# Q15:   0
# Q16:   1
# Q17:   1
# Q18:   0
# Q19:   1
# Q20:   0
# Q21:   "Although it's been a challenging few days, I must admit that my favorite moment of the weekend was when we had the group activity where everyone was encouraged to express their feelings through art. It was truly a cathartic experience, and I found it comforting to share my emotions through painting and drawing alongside others who were going through similar heartaches. It felt like we were all part of a healing community, and I could feel a sense of unity that I hadn't experienced before during my grief journey.
# It was during that art session when I had a breakthrough moment. I realized that I don't have to navigate this difficult time alone. I met an adult who shared their own experience as a survivor's child and found ways to create a new sense of purpose in their life. Their story inspired me, and I see now that we can find meaning in honoring our loved ones in our own ways while also embracing our individual growth. I am grateful for the connections and resources that TAPS provided this weekend, as they have given me hope for the future."
# Q22:   At TAPS Seminar, I connected with fellow survivors, gained emotional support and practical resources, but still struggle to find purpose daily as a surviving child; more tailored guidance is needed for young grieving family members.
# Label: 1
# Reason: Labeled as Stage 1 due to the respondent’s expression of intense emotions, a feeling of isolation, and the struggle to find daily purpose, which aligns with the initial shock and overwhelming grief of immediate loss.

# Q1:    3
# Q2:    nan
# Q3:    1
# Q4:    2
# Q5:    2
# Q6:    0
# Q7:    2
# Q8:    2
# Q9:    2
# Q10:   2
# Q11:   1
# Q12:   1
# Q13:   3
# Q14:   1
# Q15:   1
# Q16:   2
# Q17:   1
# Q18:   1
# Q19:   1
# Q20:   2
# Q21:   "I had a very emotional yet healing experience during the support group session for children. The counselor created a safe and welcoming atmosphere for all the kids to share their feelings and experiences openly. We played an interactive game that allowed us to understand the importance of acknowledging and processing our emotions together. I observed my own child as well as other children in the group opening up, and it was a rewarding moment for me as a parent. This normalized our grief and gave me hope that, together, we can continue to heal and grow."
# Q22:   nan
# Label: 3
# Reason: Labeled as Stage 3 because the response emphasizes the process of acknowledging and managing emotions in a supportive setting, which aligns with learning to process grief and gaining hope through shared healing experiences.

# Q1:    3
# Q2:    nan
# Q3:    1
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    2
# Q8:    1
# Q9:    1
# Q10:   1
# Q11:   2
# Q12:   1
# Q13:   2
# Q14:   2
# Q15:   2
# Q16:   2
# Q17:   1
# Q18:   2
# Q19:   2
# Q20:   2
# Q21:   The last kids meetin' was reely wun-hoanahlin' - so many powerful feelyings shared as the guider helped the little ones open up thur art. So proud to see ma baby speekin' thur paintbrushes and hearin' other kiddos' brav' stories. It made me feel somethin' reely deep innah my heart. That time chaynged how I do parenting for the bettah, since I got a lot of new ideas. Itz impawtant to teach my baby to feel things and share them with others. This experiance wuz truely gratefur by TAPS, they helped me see the best ways to hep ma baby. They left somethin' wunneh in my heart as a mama and I'll try to do bettah with that in mind, foah-evah.
# Q22:   nan
# Label: 3
# Reason: Labeled as Stage 3 due to the emphasis on processing emotions, learning new ways to support a child through grief, and the value of emotional expression, which aligns with learning to process and manage grief.

# Q1:    1
# Q2:    3.0
# Q3:    1
# Q4:    1
# Q5:    2
# Q6:    1
# Q7:    1
# Q8:    1
# Q9:    0
# Q10:   1
# Q11:   4
# Q12:   0
# Q13:   1
# Q14:   3
# Q15:   1
# Q16:   0
# Q17:   0
# Q18:   0
# Q19:   1
# Q20:   0
# Q21:   "During the weekend seminar, my favorite moment was when we were encouraged to share our stories with each other. It felt like a beacon of light in my otherwise dark and overwhelming journey. We were all struggling, and there was a certain comfort in knowing that we're not alone in this. It's not easy to handle everything by myself, especially as a surviving child. I feel deeply disconnected without guidance and attention from my grieving adults, but sharing my experiences with everyone at TAPS gave me some sense of purpose and connection. Though I still face many challenges, this weekend has reminded me that we're all in this together, and that is what I find immensely comforting."
# Q22:   nan
# Label: 1
# Reason: Labeled as Stage 1 due to the respondent’s expression of feeling overwhelmed, isolated, and disconnected, with a strong need for support and guidance, which aligns with the intense, initial shock and emotional state of immediate grief.

# Q1:    4
# Q2:    0.0
# Q3:    2
# Q4:    1
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    1
# Q9:    1
# Q10:   0
# Q11:   1
# Q12:   2
# Q13:   1
# Q14:   0
# Q15:   3
# Q16:   1
# Q17:   0
# Q18:   2
# Q19:   0
# Q20:   2
# Q21:   "During the weekend seminar with TAPS, my favorite moment was when we participated in the group sharing and bonding activities. It helped me realize that each family member is going through their unique grief journey and we all need to have patience and understanding for one another.
# One of my breakthrough moments was when we were given the tool to create a 'Healing Hearts Artwork' with our family. It allowed me to finally express my emotions and validate my pain in a creative way. This experience made me feel a sense of unity with my family members and helped me see that we are all in this together. It was a powerful moment that allowed me to address the tension within the family and strive for a more supportive environment moving forward."
# Q22:   nan
# Label: 2

# Q1:    4
# Q2:    nan
# Q3:    1
# Q4:    1
# Q5:    1
# Q6:    0
# Q7:    0
# Q8:    3
# Q9:    3
# Q10:   0
# Q11:   0
# Q12:   0
# Q13:   0
# Q14:   3
# Q15:   0
# Q16:   1
# Q17:   1
# Q18:   0
# Q19:   1
# Q20:   2
# Q21:   nan
# Q22:   nan
# Label: 2
# Reason: Labeled as Stage 2 due to the low scores in optimism, inner strength, and direction, combined with high anxiety about the future, suggesting struggles with family relationships and support, characteristic of navigating family dynamics in grief.

# Q1:    0
# Q2:    nan
# Q3:    2
# Q4:    2
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    0
# Q9:    2
# Q10:   2
# Q11:   1
# Q12:   2
# Q13:   2
# Q14:   1
# Q15:   1
# Q16:   2
# Q17:   3
# Q18:   2
# Q19:   1
# Q20:   2
# Q21:   "I am grateful for the opportunities provided at TAPS to reminisce on the good times with my loved one and create new memories with my Family Unit. One moment that stands out for me was the candle lighting ceremony on the last night. As we lit our candles and shared our stories, I felt an overwhelming sense of unity and support from the other families present. This experience gave me the strength to face the upcoming tough days and find solace in our shared grief journey. We also had a breakthrough moment when we participated in the art therapy session. Expressing our emotions through painting and drawing helped us to better understand our individual grief experiences, while bringing us closer as a Family Unit. Overall, the weekend gave me hope and renewed energy to face life's challenges and cherish the moments that matter with my family."
# Q22:   nan
# Label: 4
# Reason: Labeled as Stage 4 due to the focus on shared grief experiences during significant events, such as the candle lighting ceremony and art therapy, which brought unity and strength in navigating "moments that matter" with family.

# Q1:    2
# Q2:    7.0
# Q3:    1
# Q4:    2
# Q5:    1
# Q6:    2
# Q7:    0
# Q8:    1
# Q9:    2
# Q10:   2
# Q11:   2
# Q12:   1
# Q13:   1
# Q14:   3
# Q15:   2
# Q16:   2
# Q17:   1
# Q18:   2
# Q19:   1
# Q20:   1
# Q21:   nan
# Q22:   Asistir al Seminario TAPS junto a mi hijo fue una experiencia transformadora. Proporcionó un ambiente de apoyo para que procesáramos nuestro duelo, ofreciendo recursos valiosos, orientación y conexiones con otras personas enfrentando desafíos similares. El seminario mejoró significativamente nuestro viaje de sanación y altamente lo recomiendo.
# Label: 3
# Reason: Labeled as Stage 3 due to the response’s focus on processing grief, receiving support, and using resources and connections to aid in healing, aligning with learning to process emotions.

# Q1:    0
# Q2:    7.0
# Q3:    2
# Q4:    0
# Q5:    1
# Q6:    0
# Q7:    3
# Q8:    0
# Q9:    3
# Q10:   3
# Q11:   0
# Q12:   0
# Q13:   3
# Q14:   0
# Q15:   3
# Q16:   3
# Q17:   3
# Q18:   3
# Q19:   3
# Q20:   3
# Q21:   At a recent TAPS seminar, a panel discussion of grieving family members sparked a breakthrough, inspiring me to support and uplift others in their grief journey, volunteer, and contribute to the TAPS community's mission of providing love, support, and healing.
# Q22:   nan
# Label: 6
# Reason: Labeled as Stage 6 due to the response's focus on giving back, supporting others, and contributing to the TAPS community, reflecting a desire to help and mentor as part of personal healing.

# Q1:    0
# Q2:    nan
# Q3:    2
# Q4:    1
# Q5:    1
# Q6:    2
# Q7:    1
# Q8:    0
# Q9:    3
# Q10:   3
# Q11:   0
# Q12:   1
# Q13:   3
# Q14:   0
# Q15:   1
# Q16:   3
# Q17:   3
# Q18:   3
# Q19:   3
# Q20:   3
# Q21:   At the TAPS seminar, I experienced a life-changing weekend through art-based group activities and a memorial service, which inspired me to create a local support group for grieving military families.
# Q22:   The TAPS Seminar significantly impacted my life by offering valuable support, guidance, and a sense of community for survivors like me. Through meaningful connections and workshops, I discovered renewed purpose, strengthened my support system, and look forward to my continued growth with the TAPS family.
# Label: 5
# Reason: Labeled as Stage 5 due to the response's focus on finding community support, a renewed sense of purpose, and commitment to personal growth within the TAPS community.

# Q1:    4
# Q2:    nan
# Q3:    2
# Q4:    2
# Q5:    0
# Q6:    1
# Q7:    3
# Q8:    0
# Q9:    1
# Q10:   2
# Q11:   3
# Q12:   3
# Q13:   2
# Q14:   3
# Q15:   3
# Q16:   3
# Q17:   3
# Q18:   1
# Q19:   0
# Q20:   1
# Q21:   "I had a breakthrough moment at the TAPS seminar that truly stands out for me. During one of the group discussions, I finally expressed my feelings of frustration and sadness to my fellow survivors. It was in that moment of vulnerability that I realized I wasn't alone in my struggles. We all shared similar experiences, and that brought a sense of relief and comfort to me. It also helped me to understand that my family members are also going through their unique grief journeys.

# The support and understanding from the other seminar participants allowed me to see things from their perspective and have a more empathetic approach when interacting with them. As a result, I feel like the tension within our family has eased a bit, and we're learning how to support each other better. I am grateful for the opportunity to connect with others who truly understand what we are going through as survivors. This seminar and its community will always hold a special place in my heart."
# Q22:   "I appreciated the TAPS Seminar as it provided a safe space for me to process my grief and connect with others who have gone through similar experiences. However, I found the lack of support from some family members during the seminar difficult to navigate. It seems that they were not at the same stage of grief as me, and this created tension among us. While the TAPS team offered valuable tools and resources for coping with grief, I believe they could have provided more guidance on addressing these underlying family dynamics. Overall, though, my experience at the seminar was insightful and I am grateful for the support of my fellow seminar attendees who helped me during this challenging journey."
# Label: 2

# Q1:    6
# Q2:    nan
# Q3:    0
# Q4:    2
# Q5:    1
# Q6:    2
# Q7:    0
# Q8:    3
# Q9:    3
# Q10:   3
# Q11:   3
# Q12:   1
# Q13:   3
# Q14:   2
# Q15:   0
# Q16:   3
# Q17:   1
# Q18:   1
# Q19:   0
# Q20:   2
# Q21:   "In sessions promoting open communitcation, we learnt to acknowledge and undarstand our unique grief experienses, finding support with each other whilst celebrating our love ones' legacies during the remembrance ceremony, leading to uniting and sharid healing."
# Q22:   "I attended the TAPS Seminar, and overall, I found the experience to be both thought-provoking and supportive. It provided me with valuable tools and resources to help navigate my grief journey. However, one area where I felt some challenges was in the dynamics within my family unit. While I was able to connect with other attendees and receive their empathetic understanding, I discovered differing grief experiences and perceptions among my family members.
# I appreciated the opportunity to attend workshops and activities focused on grief and self-care, as it allowed me to gain a better understanding of my own emotions and needs. However, I noticed that the lack of support from some family members made it harder for me to cope with the emotional aspects of the seminar. It highlighted the ongoing tension within our family and the continued need for open communication and understanding.
# Despite this challenge, I am grateful for the connections I made during the seminar and the valuable resources provided by TAPS experts. I hope to continue engaging with the TAPS community, as I believe it has the potential to help strengthen the support system within my family and further my own healing process."
# Label: 2
# Reason: Labeled as Stage 2 due to emphasis on family dynamics, differing grief experiences, and need for open communication.

# Q1:    3
# Q2:    1.0
# Q3:    1
# Q4:    1
# Q5:    1
# Q6:    1
# Q7:    2
# Q8:    2
# Q9:    0
# Q10:   1
# Q11:   3
# Q12:   1
# Q13:   0
# Q14:   3
# Q15:   0
# Q16:   0
# Q17:   0
# Q18:   0
# Q19:   2
# Q20:   1
# Q21:   nan
# Q22:   nan
# Label: 1
# Reason:It reflects intense feelings of isolation, fear about the future, and an absence of positivity or purpose, aligning with the overwhelmed and uncertain state characteristic of immediate grief.

# Q1:    1
# Q2:    7.0
# Q3:    2
# Q4:    1
# Q5:    1
# Q6:    2
# Q7:    1
# Q8:    0
# Q9:    3
# Q10:   3
# Q11:   0
# Q12:   3
# Q13:   3
# Q14:   0
# Q15:   3
# Q16:   3
# Q17:   3
# Q18:   3
# Q19:   3
# Q20:   3
# Q21:   Example response: "Attending the TAPS seminar over the weekend provided me with a sense of belonging and strengthened my connection to the surviving family unit. My favorite moment came during the group discussion, where we shared our stories and experiences. We were all in a similar situation, and I felt a deep sense of comfort knowing that I'm not alone in this journey.

# The breakthrough moment for me was when the trauma expert emphasized the importance of finding new purpose and setting goals to help us move towards positive integration. This resonated with me, and I realized that I need to focus on my own personal growth and well-being in order to continue honoring the memory of my loved one. This weekend has given me the motivation and support to find that new purpose and connect with this wonderful community. Thank you, TAPS, for providing this invaluable service."
# Q22:   nan
# Label: 5
# Reason: This response is classified as Stage 5 becuase in the open-ended questions, despite mentioning of connections and a new sense of purpose, the respondent does not mention giving back or readiness to support others.

# Q1:    7
# Q2:    1.0
# Q3:    2
# Q4:    1
# Q5:    2
# Q6:    2
# Q7:    0
# Q8:    2
# Q9:    3
# Q10:   3
# Q11:   0
# Q12:   3
# Q13:   3
# Q14:   0
# Q15:   3
# Q16:   3
# Q17:   3
# Q18:   3
# Q19:   3
# Q20:   3
# Q21:   Durante el seminario de fin de semana, tuve una experiencia profunda y emocional al conectarme con otras familias que también navegaban por los desafíos del duelo y la pérdida. La discusión en panel sobre redescubrir propósito y sentido en el after de la pérdida realmente me resonó, y me ofreció valiosos insights que me animaron a reflexionar sobre la dirección de mi propia vida. Esta discusión no solo resaltó la importancia de forjar conexiones con aquellos que comparten y entienden nuestras experiencias, sino que también me impulsó a considerar adoptar una nueva perspectiva en mi viaje.
# Uno de los puntos de inflexión del seminario fue el taller sobre crear un legado viviente para nuestros seres queridos. A través de este taller, obtuve claridad y motivación renovada para establecer un proyecto comunitario dedicado a honrar la memoria de mi ser amado de una manera significativa y positiva. Al trabajar en colaboración con individuos que comparten un viaje similar, podemos apoyarnos mutuamente, crear un espacio seguro para conversaciones emotivas y garantizar que la memoria de nuestros seres queridos continúe brillando en nuestras vidas y en la vida de los demás.
# Como resultado de mi participación en este seminario de fin de semana, he desarrollado un vínculo indisolvable con la comunidad TAPS y estoy increíblemente agradecido por su apoyo inquebrantable y las enriquecedoras perspectivas que me han compartido. Este seminario ha abierto mis ojos a nuevas oportunidades de crecimiento, sanación y conexión, y estoy emocionado por emprender este viaje con mi comunidad recién formada.
# Q22:   "Encontré el Seminario TAPS como una experiencia realmente transformadora. Me proporcionó un sentimiento de conexión y pertenencia dentro de una comunidad que entiende mi dolor único. Los talleres y recursos ofrecidos me han dado la fuerza y la orientación para emprender mi viaje hacia una integración positiva. No solo adquirí herramientas para gestionar mi duelo, sino que también establecí conexiones significativas con personas que tienen una historia similar. Estoy agradecido por el apoyo y la validación que recibí en el seminario, y me siento empoderado para seguir adelante con un nuevo sentimiento de propósito."
# Label: 5
# Reason: This response is classified as Stage 5 becuase in the open-ended questions, despite mentioning of connections and a new sense of purpose, the respondent does not mention giving back or readiness to support others.

# Q1:    0
# Q2:    7.0
# Q3:    2
# Q4:    0
# Q5:    1
# Q6:    1
# Q7:    1
# Q8:    0
# Q9:    3
# Q10:   3
# Q11:   0
# Q12:   3
# Q13:   3
# Q14:   0
# Q15:   3
# Q16:   3
# Q17:   3
# Q18:   3
# Q19:   3
# Q20:   3
# Q21:   nan
# Q22:   "I went to TAPS seminar becuz I am a healthy n proactive partipicipant in the grief journey. Duaring the weekend, it helped me a lotta to connect wit othr survivors, we talked bout our experenses. I tink TAPS combo of peer support and expert guidens is uniq n valuab to heeling.
# I left the seminar feelin empowerd, ready to help in TAPS communti. I wanna teech, volunteeer in charity drives n evnts and lend a hand. It is impotant to giv bak to TAPS is pa of my mouring process. I belive it can help TAPS gro and wel comtunity.
# Ovaoll, I strongli recomd TAPS seminar to any1 who is grieving the los of a loved one that servd in US militry. The exprience was lif chanjin for me. I hop by talkin bout my stori n engagin othr partipicpants, I can helot her find comfort n communit. It did for me."
# Label: 6
# Reason: Labeled as Stage 6 due to the respondent's focus on giving back, volunteering, and supporting others in the TAPS community, showing a proactive approach to help and empower others as part of their healing journey.---------------------
# """

In [15]:
def classify_response(response):
    try:
        messages = [
            {
                "role": "system",
                "content": """
                            You are Eva, an AI expert in survivor care at The Tragedy Assistance Program for Survivors (TAPS).
                            You are tasked with mapping survivor survey responses to appropriate stages (or "epochs") within the TAPS survivor journey map
                            """
            },
            {
                "role": "user",
                "content": """
                        TAPS has developed an initial version of what they call a "Journey Map", which maps the major stages and tasks associated with a survivor's grief journey. They use this journey map to help guide the appropriate care for survivors in their programs. The major stages identified in this journey map are:

                        1. "Immediate Grief, Shock & Emotion": Overwhelmed, loss of purpose; shock and trauma emotions (isolation) present and challenging to understand. Individuals may struggle to deal with family responsibilities alone. Surviving Child: Feeling disconnected without guidance and attention from grieving adults.
                        2. "Navigating Family Relationships": Experiencing tension between individuals within the family unit; lack of support from family members. Surviving Family Unit: Perception of other family members’ grief experience. Each family member may be at different phases of their grief journey.
                        3. "Learning to Process Grief": Experiencing grief and learning to process those emotions. Surviving Child: Seeks guidance and acknowledgment of grief; benefit from opportunities to open up and process with kids in similar situations to normalize emotions.
                        4. "Moments That Matter": Renewed experience of grief around anniversaries of loss, holidays, and special moments. Surviving Family Unit: Navigating special moments (sports, school achievements, moments that matter).
                        5. "Feeling Immersed, Connected & Seen": Finding new purpose and goals to begin moving towards Positive Integration. Surviving Family Unit: Connected to a broader community; support system; not the only person/family experiencing loss.
                        6. "New Growth & Purpose": Healthy point in grief journey; feeling capable to help others and a desire to do so. Surviving Family Unit: Ready to give back to the TAPS community through mentorship programs, volunteering at charity drives & events, etc.

                        The survey questions that are used to map stages are listed below:
                        -----------------------------------------------
                        1. Please share all of the reasons you chose to attend the TAPS [SEMINAR NAME]
                        I had attended a previous Regional or Nationals seminar
                        To connect with other survivors who share a similar loss
                        To learn more about resources TAPS has to offer
                        To learn new tools and information to help me with my grief
                        To learn more about how to support my adult family members in their grief
                        To learn more about how to support my child(ren) in their grief
                        For my child(ren) to attend Good Grief Camp
                        For my child(ren) to connect with a Military Mentor

                        2. How did you find out about the TAPS [SEMINAR NAME]?
                        I found this event while searching for grief resources
                        I attended a seminar last year and had already marked my calendar!
                        TAPS invited me to this event via email
                        My TAPS Survivor Care Team Member invited me
                        My Peer Mentor or another survivor invited me
                        Through the TAPS website
                        Through the TAPS magazine
                        Through a TAPS Social Media Page

                        3. This seminar helped me to feel socially connected
                        Disagree
                        Neither agree nor disagree
                        Agree

                        4. This seminar taught me new ways to cope with my grief
                        Disagree
                        Neither agree nor disagree
                        Agree

                        5. This seminar helped me to better understand my grief
                        Disagree
                        Neither agree nor disagree
                        Agree

                        6. This seminar has given me hope for the future
                        Disagree
                        Neither agree nor disagree
                        Agree

                        7. Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to connect with TAPS virtually / at in-person programs?
                        Extremely likely / Very likely
                        Somewhat likely
                        Neither likely nor unlikely / Neutral
                        Somewhat unlikely
                        Extremely unlikely / Not at all likely

                        8. Based on your experience at the TAPS [SEMINAR NAME], how likely are you to continue to bring your child(ren) to connect with TAPS at in-person programs?
                        Extremely likely / Very likely
                        Somewhat likely
                        Neither likely nor unlikely / Neutral
                        Somewhat unlikely
                        Extremely unlikely / Not at all likely

                        9. I have a positive outlook toward life
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        10. I have short and/or long range goals
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        11. I feel all alone
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        12. I can see possibilities in the midst of difficulties
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        13. I have faith that gives me comfort
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        14. I feel scared about my future
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        15. I can recall happy/joyful times
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        16. I have deep inner strength
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        17. I am able to give and receive care/love
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        18. I have a sense of direction
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        19. I believe that each day has potential
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        20. I feel my life has value and worth
                        Strongly Disagree
                        Disagree
                        Agree
                        Strongly Agree

                        21. Please share with TAPS your favorite moment of the weekend? Did you have a breakthrough moment this weekend you would like to share?

                        22. Please share any additional feedback you have regarding your TAPS Seminar experience.
                        -----------------------------------------------
                        
                        I will give you a survey response and you will analyse it, step-by-step, to know which stage the respondent belongs to. Got it?
                        """
            },
            {
                "role": "assistant",
                "content": "Yes, I understand. I am Eva, and I will analyse your survey response."
            },

            {
                "role": "user",
                "content": f"""Perfect! Here are some examples for you to learn, pay attention to how you can distingush stages that are close(3 and 4, 5 and 6):
                {examples}
            """
            },

            {
                "role": "assistant",
                "content": "Thank you for the examples provided. I will learn from the example to improve my analysis"
            },

            {
                "role": "user",
                "content": f"""Great! Finally, you will answer following this template:
                            Answer: The respondnet is in Stage [stage number here]. Reasoning step 1:\nReasoning step 2:\nReasoning step 3:\n
                            """
            },

            {
                "role": "assistant",
                "content": "Yes, I understand. I will follow the template."
            },

            {
                "role": "user",
                "content": f"""Great,  Let's begin then :)
                            For the given response:
                            {response}
                            What stage is the respondent currently experiencing?
                            Answer: Let's think step by step to reach the right
                            conclusion."""
            }
        ]

        result = client.chat.completions.create(
                model="Hermes-3-Llama-3.1-70B",
                messages=messages,
                max_tokens = 1000,
                temperature = 0.1
            )

        for char in result['choices'][0]['message']['content'].strip():
            if char.isdigit():
                first_digit = char
                break

        time.sleep(10)
        return int(first_digit)
    except Exception as e:
        print(f"Error: {e}. Skipping this response.")
        return None  # Return None or a specific default value for failed cases

In [16]:
test_data = data.iloc[[0,  3,  4,  5,  6,  8,  9, 12, 13, 15, 16, 17, 24, 25, 26, 27, 30, 31,
       32, 33, 34, 37, 39, 41, 44, 45, 47]].reset_index(drop=True)

In [17]:
# labels = []
# i = 0
# for response in test_data['formatted_response']:
#     labels.append(classify_response(response))
#     if test_data['label'][i] == labels[i]:
#         print(f'{i+1} done, correct')
#     else:
#         print(f'{i+1} done, incorrect')
#         print(test_data['label'][i], '-->', labels[i])
#     i = i+1

# test_data['predicted_label'] = labels

In [18]:
# # prompt: compute accuracy using label and predicted label in test_data

# from sklearn.metrics import accuracy_score, f1_score

# # Assuming test_data has 'label' and 'predicted_label' columns
# y_true = test_data['label']
# # y_pred = test_data['majority_label']
# y_pred = test_data['predicted_label'].fillna(0)

# accuracy = accuracy_score(y_true, y_pred)
# print(f"Accuracy: {accuracy}")

In [19]:
# f1_score(y_true, y_pred, average='weighted')

In [20]:
# test_data[['label', 'predicted_label']]

In [21]:
# mismatched_rows = test_data[test_data['label'] != test_data['predicted_label']]
# mismatched_rows

In [22]:
# mismatched_rows.iloc[1]['q21']

In [23]:
test = pd.read_csv('test.csv')

In [24]:
test.shape

(2950, 23)

In [25]:
test['formatted_response'] = test.apply(format_response, axis=1)

In [26]:
save_path = 'test_results.csv'

In [29]:
if os.path.exists(save_path):
    test = pd.read_csv(save_path)
else:
    test['predicted_label'] = None  # Initialize column if not already in the CSV

while test['predicted_label'].isna().any():
    for idx, row in test.iterrows():
        if pd.isna(row['predicted_label']):  # Skip if label already exists
            label = classify_response(row['formatted_response'])
            test.at[idx, 'predicted_label'] = label

        # Save progress every 10 rows
        if idx % 10 == 0:
            test.to_csv(save_path, index=False)
            print(f"Progress saved at row {idx}")
    test.to_csv(save_path, index=False)
    print("Round completed. Checking for unclassified rows...")

# Final save after loop completion
test.to_csv(save_path, index=False)
print("Classification completed and progress saved.")

Progress saved at row 0
Progress saved at row 10
Progress saved at row 20
Progress saved at row 30
Progress saved at row 40
Progress saved at row 50
Progress saved at row 60
Progress saved at row 70
Progress saved at row 80
Progress saved at row 90
Progress saved at row 100
Progress saved at row 110
Progress saved at row 120
Progress saved at row 130
Progress saved at row 140
Progress saved at row 150
Progress saved at row 160
Progress saved at row 170
Progress saved at row 180
Progress saved at row 190
Progress saved at row 200
Progress saved at row 210
Progress saved at row 220
Progress saved at row 230
Progress saved at row 240
Progress saved at row 250
Progress saved at row 260
Progress saved at row 270
Progress saved at row 280
Progress saved at row 290
Progress saved at row 300
Progress saved at row 310
Progress saved at row 320
Progress saved at row 330
Progress saved at row 340
Progress saved at row 350
Progress saved at row 360
Progress saved at row 370
Progress saved at row 3

KeyboardInterrupt: 

In [ ]:
results = test[['Id', 'predicted_label']]

results.columns = ['Id', 'Label']

results.to_csv('submission.csv', index = False)

In [ ]:
results